# Getting Started

In this tutorial, you will know how to
- use the models in **tatk** to build a dialog agent.
- build a simulator to chat with the agent and evaluate the performance.
- try different module combinations.

Let's get started!

## Environment Setup
Run the command below to install tatk for once. Then restart the notebook and ignore this commend.

In [0]:
# first install tatk and restart the notebook
! rm -rf tatk && git clone https://github.com/thu-coai/tatk.git && cd tatk && pip install -e .

## Build an agent

We use the models adapted on [Multiwoz dataset](https://www.aclweb.org/anthology/D18-1547) to build our agent. This pipeline agent consists of NLU, DST, Policy and NLG modules.

First, import some models:

In [0]:
import sys
import os
# Agent
from tatk.dialog_agent import PipelineAgent, BiSession
# common import: tatk.$module.$model.$dataset
from tatk.nlu.svm.multiwoz import SVMNLU
from tatk.nlu.bert.multiwoz import BERTNLU
from tatk.dst.rule.multiwoz import RuleDST
from tatk.policy.rule.multiwoz import Rule
from tatk.nlg.template.multiwoz import TemplateNLG
from tatk.evaluator.multiwoz_eval import MultiWozEvaluator
import random
import numpy as np
from pprint import pprint

Then, create the models and build an agent:

In [2]:
# svm nlu trained on usr sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/svm/multiwoz` for more information 
sys_nlu = SVMNLU(mode='usr')
# simple rule DST
sys_dst = RuleDST()
# rule policy
sys_policy = Rule(character='sys')
# template NLG
sys_nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg)

Load from model_file param
loading saved Classifier
loaded.


That's all! Let's chat with the agent using its `response` function:

sys_agent.response("I want to find a moderate hotel")

In [4]:
sys_agent.response("Which type of hotel is it ?")

'It is a guesthouse .'

In [5]:
sys_agent.response("OK , where is its address ?")

'Their address in our system is listed as pool way, whitehill road, off newmarket road .'

In [6]:
sys_agent.response("Thank you !")

'Welcome , it was a pleasure serving you .'

In [7]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

'There are 3 restaurants in that area that fit that criteria . Excellent . the lucky star is just your thing . Okay , may i suggest chinese food ?'

In [8]:
sys_agent.response("Which kind of food it provides ?")

'That is a chinese restaurant .'

In [9]:
sys_agent.response("Book a table for 5 , this Sunday .")

'Here is the booking information : booking was successful . reference number is : 00000003.'

## Build a Simulator to Chat with the Agent and Evaluate

In many one-to-one task-oriented dialog system, a simulator is essential to train an RL agent. In our framework, we doesn't distinguish user or system, all speakers are **agents**. The simulator is also an agent, with specific policy inside for accomplishing the user goal.

We use Agenda policy for the simulator, this policy requires dialog act input, which means we should set DST argument of `PipelineAgent` to `None`. Then the `PipelineAgent` will pass dialog act to policy directly. Refer to `PipelineAgent` doc for more details.

In [10]:
# bert nlu trained on sys sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/bert/multiwoz` for more information 
user_nlu = BERTNLU(mode='sys')
# not use dst
user_dst = None
# rule policy
user_policy = Rule(character='usr')
# template NLG
user_nlg = TemplateNLG(is_user=True)
# assemble
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg)

load train, size 8434
load val, size 999
load test, size 1000
loaded train, size 56750
loaded val, size 7365
loaded test, size 7372
dialog act num: 34
sentence label num: 64
tag num: 312
Load from model_file param
Load from /content/tatk/tatk/nlu/bert/multiwoz/output/sys/bestcheckpoint.tar
train step 29900
BERTNLU loaded
Loading goal model is done


Now we have a simulator and an agent. we will use an existed simple one-to-one conversation controller `BiSession`, you can also define your own Session class for your special need. 

We add `MultiWozEvaluator` to evaluate the performance. It uses the parsed dialog act input and policy output dialog act to calculate **inform f1**, **book rate**, and whether the task is **success**.

In [0]:
evaluator = MultiWozEvaluator()
sess = BiSession(sys_agent=sys_agent, user_agent=user_agent, kb_query=None, evaluator=evaluator)

Let's make this two agents chat! The key is `next_turn` method of Session class.

In [12]:
random.seed(20190827)
np.random.seed(20190827)
sys_response = ''
sess.init_session()
print('init goal:')
pprint(sess.evaluator.goal)
print('-'*50)
for i in range(40):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print()
    if session_over is True:
        print('task success:', sess.evaluator.task_success())
        print('book rate:', sess.evaluator.book_rate())
        print('inform precision/recall/f1:', sess.evaluator.inform_F1())
        print('-'*50)
        print('final goal:')
        pprint(sess.evaluator.goal)
        print('='*100)
        break

init goal:
{'attraction': {'info': {'area': 'centre', 'type': 'college'},
                'reqt': {'entrance fee': '?', 'phone': '?'}},
 'train': {'book': {'people': '7'},
           'booked': '?',
           'info': {'day': 'wednesday',
                    'departure': 'cambridge',
                    'destination': 'london kings cross',
                    'leaveAt': '12:15'},
           'reqt': {'duration': '?'}}}
--------------------------------------------------
user: Do you have any college attractions. I am also looking for places to go in town . maybe something in the centre .
sys: There are 44 , anything in particular you are looking for ? I 'd recommend the fez club . would you like some information on it ?

user: Can you give me their phone number please ? Yes , what are the entrance fees ?
sys: Its entrance fee is ? . The phone number is 01223300085 .

user: I just need to know how much the entrance fee is .
sys: The park is ? .

user: Okay , are there any colleges in the c

`BiSession` allows two agents chat in dialog act level or natural language level, once the input and output are consistent. Example configurations:

| usr input ==>       | usr NLU | usr DST | usr Policy | usr NLG  | ==> sys input ==>        | sys NLU | sys DST | sys Policy | sys NLG  |
| ---------------- | ------- | ------- | ---------- | -------- | ---------------- | ------- | ------- | ---------- | -------- |
| Dialog act       | None    | Rule    | Rule       | None     | Dialog act       | None    | None    | Rule       | None     |
| Natural language | Bert    | Rule    | Rule       | None     | Dialog act       | None    | None    | Rule       | Template |
| Dialog act       | None    | Rule    | Rule       | Template | Natural language | SVM     | None    | Rule       | None     |
| Natural language | Bert    | Rule    | Rule       | Template | Natural language | SVM     | None    | Rule       | Template |


We have tried the last configuration before. Let's try the second configuration.


In [0]:
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, None)
sys_agent = PipelineAgent(None, sys_dst, sys_policy, sys_nlg)
evaluator = MultiWozEvaluator()
sess = BiSession(sys_agent=sys_agent, user_agent=user_agent, kb_query=None, evaluator=evaluator)

In [14]:
random.seed(20190827)
np.random.seed(20190827)
sys_response = ''
sess.init_session()
print('init goal:')
pprint(sess.evaluator.goal)
print('-'*50)
for i in range(40):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print()
    if session_over is True:
        print('task success:', sess.evaluator.task_success())
        print('book rate:', sess.evaluator.book_rate())
        print('inform precision/recall/f1:', sess.evaluator.inform_F1())
        print('-'*50)
        print('final goal:')
        pprint(sess.evaluator.goal)
        print('='*100)
        break

init goal:
{'attraction': {'info': {'area': 'centre', 'type': 'college'},
                'reqt': {'entrance fee': '?', 'phone': '?'}},
 'train': {'book': {'people': '7'},
           'booked': '?',
           'info': {'day': 'wednesday',
                    'departure': 'cambridge',
                    'destination': 'london kings cross',
                    'leaveAt': '12:15'},
           'reqt': {'duration': '?'}}}
--------------------------------------------------
user: {'Attraction-Inform': [['Type', 'college'], ['Area', 'centre']]}
sys: There are 13 , anything in particular you are looking for ? I 'd recommend emmanuel college . would you like some information on it ?

user: {'Attraction-Request': [['Phone', '?'], ['Fee', '?']]}
sys: The phone number is 01223334900 . Their entrance fee is free by our system currently .

user: {'Train-Inform': [['Depart', 'cambridge'], ['Day', 'wednesday'], ['Leave', '12:15'], ['Dest', 'london kings cross']]}
sys: Would you like me to book you on t

After removing user NLG and system NLU, the conversation is more efficient.

## Try Different Module Combinations

The combination modes of pipeline agent modules are flexible. We support joint model such as [MDBT](https://www.aclweb.org/anthology/P18-2069) (NLU+DST) and [MDRG](https://pdfs.semanticscholar.org/47d0/1eb59cd37d16201fcae964bd1d2b49cfb55e.pdf) (Policy+NLG), once the input and output are matched with previous and next module. We also support End2End model such as [Sequicity](https://www.comp.nus.edu.sg/~kanmy/papers/acl18-sequicity.pdf).

### MDBT
- NLU: None
- DST: MDBT
- Policy: Rule
- NLG: TemplateNLG

In [0]:
from tatk.dst.mdbt.multiwoz.mdbt import MultiWozMDBT
nlu = None
# simple rule DST
dst = MultiWozMDBT()
# rule policy
policy = Rule()
# template NLG
nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(nlu, dst, policy, nlg)

### MDRG

- NLU: SVM
- DST: Rule
- Policy: MDRG
- NLG: None

In [0]:
from tatk.policy.mdrg.multiwoz.policy import MDRGWordPolicy
# svm nlu trained on usr sentence of multiwoz
# go to README.md under `tatk/tatk/nlu/svm/multiwoz` for more information 
nlu = SVMNLU(mode='usr')
# simple rule DST
dst = RuleDST()
# rule policy
policy = MDRGWordPolicy()
# template NLG
nlg = None
# assemble
sys_agent = PipelineAgent(nlu, dst, policy, nlg)

### Sequicity

Sequicity inherits from interface `Agent` directly.

In [0]:
from tatk.e2e.sequicity.multiwoz import Sequicity
sequicity = Sequicity()
sys_agent = sequicity